In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import zipfile
import os

zip_ref = zipfile.ZipFile('/content/drive/MyDrive/DataSet/CatAndDog.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall('/tmp') #Extracts the files into the /tmp folder
zip_ref.close()

In [ ]:
import os
len(os.listdir('/tmp/data'))

3

In [ ]:
#Directory with our training horse pictures
train_cat_dir = os.path.join('/tmp/data/train/cats')
#Directory with our training human pictures
train_dog_dir = os.path.join('/tmp/data/train/dogs')
#Directory with our training horse pictures
validation_cat_dir = os.path.join('/tmp/data/validation/cats')
#Directory with our training horse pictures
validation_dog_dir = os.path.join('/tmp/data/validation/dogs')

In [ ]:
train_cat_names = os.listdir(train_cat_dir)
train_dog_names = os.listdir(train_dog_dir)

validation_cat_hames = os.listdir(validation_cat_dir)
validation_dog_names = os.listdir(validation_dog_dir)

Building up the Model

In [ ]:
import tensorflow as tf

In [ ]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])

In [ ]:
from tensorflow.keras.preprocessing.image  import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        '/tmp/data/train/',  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=128,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Flow training images in batches of 128 using train_datagen generator
validation_generator = validation_datagen.flow_from_directory(
        '/tmp/data/validation/',  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=32,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

Found 20000 images belonging to 2 classes.
Found 4800 images belonging to 2 classes.


In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=8,
    epochs=15,
    verbose=1,
    validation_data = validation_generator,
    validation_steps=8)

Epoch 1/15
8/8 [==============================] - 35s 579ms/step - loss: 0.7118 - accuracy: 0.5248 - val_loss: 0.6934 - val_accuracy: 0.4844
Epoch 2/15
8/8 [==============================] - 4s 566ms/step - loss: 0.6920 - accuracy: 0.5268 - val_loss: 0.6948 - val_accuracy: 0.4336
Epoch 3/15
8/8 [==============================] - 4s 568ms/step - loss: 0.7005 - accuracy: 0.5299 - val_loss: 0.6927 - val_accuracy: 0.4648
Epoch 4/15
8/8 [==============================] - 4s 571ms/step - loss: 0.6912 - accuracy: 0.5370 - val_loss: 0.6962 - val_accuracy: 0.4844
Epoch 5/15
8/8 [==============================] - 4s 553ms/step - loss: 0.6966 - accuracy: 0.4860 - val_loss: 0.6812 - val_accuracy: 0.5938
Epoch 6/15
8/8 [==============================] - 4s 558ms/step - loss: 0.6966 - accuracy: 0.5480 - val_loss: 0.6907 - val_accuracy: 0.5234
Epoch 7/15
8/8 [==============================] - 4s 561ms/step - loss: 0.6872 - accuracy: 0.5339 - val_loss: 0.6831 - val_accuracy: 0.6484
Epoch 8/15
8/8 [===

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys() :

  #predicting images
  path = '/content/' + fn
  img = image.load_img(path,target_size=(150,150))
  x = image.img_to_array(img)
  x = np.expand_dims(x,axis=0)

  images = np.vstack([x])
  classes = model.predict(images,batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a dog")
  else:
    print(fn + " is a cat")  

Saving cat.12400.jpg to cat.12400.jpg
Saving cat.12401.jpg to cat.12401.jpg
Saving cat.12402.jpg to cat.12402.jpg
Saving cat.12403.jpg to cat.12403.jpg
Saving cat.12404.jpg to cat.12404.jpg
Saving cat.12405.jpg to cat.12405.jpg
Saving cat.12406.jpg to cat.12406.jpg
Saving cat.12407.jpg to cat.12407.jpg
Saving cat.12408.jpg to cat.12408.jpg
[1.]
cat.12400.jpg is a dog
[1.01750786e-16]
cat.12401.jpg is a cat
[0.]
cat.12402.jpg is a cat
[1.2968071e-18]
cat.12403.jpg is a cat
[0.]
cat.12404.jpg is a cat
[1.]
cat.12405.jpg is a dog
[7.1472833e-22]
cat.12406.jpg is a cat
[9.2380606e-29]
cat.12407.jpg is a cat
[0.]
cat.12408.jpg is a cat
